In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime 
from webdriver_manager.chrome import ChromeDriverManager
import urllib.parse
import re
import bs4
import json
import time
import random
import os
import numpy as np
from tqdm import tqdm

In [15]:
##オプション##########################
#100はやりすぎかも
MIN_COUNT = 50
KEYWORD = "弾き語りさんと繋がりたい"
#弾き語りさんと繋がりたい
#####################################

## Instagramログイン関数

In [16]:
# twitterログイン
def do_login(driver):
    # ログインURL
    DOMAIN_BASE = "https://www.instagram.com/"
    login_url = DOMAIN_BASE + "accounts/login/"
    driver.get(login_url)
     
    # 電話、メールまたはユーザー名のinput要素が読み込まれるまで待機（最大10秒）
    elem_id = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.NAME, "username"))
    )
      
    try:
        # パスワードのinput要素
        elem_password = driver.find_element_by_name("password")
    
        if elem_id and elem_password:
            # ログインID入力
            time.sleep(2)
            elem_id.send_keys(LOGIN_ID)
          
            # パスワード入力
            time.sleep(2)
            elem_password.send_keys(PASSWORD)
              
      
            # ログインボタンクリック
            time.sleep(2)
            elem_btn = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, '//*[@id="loginForm"]/div/div[3]/button'))
            )
             
            actions = ActionChains(driver)
            actions.move_to_element(elem_btn)
            actions.click(elem_btn)
            actions.perform()
 
            # 適当（3秒間待つように対応しています）
            time.sleep(3)
 
            # 遷移
            # 遷移後のURLでログイン可否をチェック
            perform_url = driver.current_url
              
            if perform_url.find(login_url) == -1:
                # ログイン成功
                return True
            else:
                # ログイン失敗
                return False
               
        else:
            return False
    except:
        return False 

## スクレイピング関数

In [17]:
# driver取得
def get_driver():
      
    #　ヘッドレスモードでブラウザを起動
    options = Options()
    #options.add_argument('--headless')
    options.add_argument("--user-data-dir=" + PROFILE_PATH)
        
    # ブラウザーを起動
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options) 
    #driver = webdriver.Chrome(CHROMEDRIVER, options=options)
      
    return driver

In [18]:
# 対象ページ取得
def get_text_from_target_page(driver, first_flg, url):
    
    #最大三回ループ
    for x in range(0, 3):
        try:
            # ターゲット
            driver.get(url)
            driver.implicitly_wait(10)

            if first_flg:    
                # articleタグが読み込まれるまで待機（最大300秒）
                WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.TAG_NAME, 'article')))
        except:
            print('get_text_from_target_page エラーのため処理中断')
            if x == 2:
                return 'エラー'
                break
        else:
            break
    
    return driver.page_source

In [19]:
# 正規表現で値を抽出
def get_search_value(ptn, str):
      
    result = re.search(ptn, str)
      
    if result:
        return result.group(1)
    else:
        return None
     

In [20]:
# info取得
def get_info_from_text(text):
    INSTAGRAM_DOMAIN = "https://www.instagram.com"
    soup = bs4.BeautifulSoup(text, features='lxml')
      
    try:
        info = {}
        # 投稿（v1Nh3 kIKUG  _bz0w）
        elems = soup.find_all(class_="v1Nh3")
         
        for elem in elems:
            a_elem = elem.find("a")
            href = a_elem["href"]    
            url = INSTAGRAM_DOMAIN + href
            post_id = get_search_value("\/p\/(.*)\/", href)
             
            # img情報
            img_elem = elem.find("img")
            #alt = img_elem["alt"]
             
            post_dic = {}
            post_dic["url"] = url
            #post_dic["alt"] = alt
             
            info[post_id] = post_dic
              
        return info
         
    except:
        return None

In [21]:
# 最後の要素までスクロール
def scroll_to_elem(driver, footer_move_flg):
     
    try:
        if footer_move_flg:
            last_elem = driver.find_element_by_id("fb-root")   
 
 
            actions = ActionChains(driver);
            actions.move_to_element(last_elem);
            actions.perform();
        else:
            # 最後の要素の一つ前までスクロール
            elems = driver.find_elements_by_class_name("weEfm")
            last_elem = elems[-1]
              
            actions = ActionChains(driver);
            actions.move_to_element(last_elem);
            actions.perform();
          
        return True
    except:
        return False

In [22]:
# 投稿件数取得
def get_post_count(text):
    try:
        json_str = get_search_value("window._sharedData = (.*);<\/script>", text)
        dict = json.loads(json_str)
        post_count = dict["entry_data"]["TagPage"][0]["graphql"]["hashtag"]["edge_hashtag_to_media"]["count"]
        return post_count
    except:
        return MIN_COUNT

## 実行Script

In [23]:
import warnings
#ワーニング譜表示
warnings.simplefilter('ignore')

In [24]:
######設定###########################
CHROMEDRIVER = "./chromedriver.exe"
LOGIN_ID = "your acountID"
PASSWORD = "password"
PROFILE_PATH = 'C:/Users/hoge/AppData/Local/Google/Chrome/User Data'
#####################################

#chrome側でログイン情報保存済の場合は起動必要なし

# # ブラウザーを起動
# driver = get_driver()
# time.sleep(5)

# # ログイン
# #do_login(driver)
# time.sleep(random.uniform(2, 5))

### SNSログイン→検索ワードでスクレイピング

In [25]:

if __name__ == '__main__':
     
    # url
    url = f"https://www.instagram.com/explore/tags/{KEYWORD}/"
 
    # ブラウザーを起動
    driver = get_driver()
    time.sleep(5)
    
    # ログイン
    #do_login(driver)
    time.sleep(random.uniform(2, 5))
 
    # 対象ページのhtmlソース取得
    text_0 = get_text_from_target_page(driver, True, url)
    post_count = get_post_count(text_0)
    print("合計：" + str(post_count))
     
    info_all = {}
    count_info = 0
    buf_count_info = 0
    
    #MIN_COUNTまでループ
    while count_info < MIN_COUNT:

        # スクロール後対象ページのhtmlソース取得
        text_1 = driver.page_source
        info = get_info_from_text(text_1)

        if not None:
            info_all.update(info)

        count_info = len(info_all)
        time.sleep(1)
        print(count_info)

        if buf_count_info==count_info:
            time.sleep(3)

        result_flg = scroll_to_elem(driver, False)
        buf_count_info = count_info

        if post_count <= count_info:
            break
            
time.sleep(5)



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\kohiz\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


合計：50
21
33
33
60


### Insstagramでフォローする

In [ ]:
##追加検討###########################
#いいね取り消しや既いいね判断
####################################


follower = []
follow = []

#確立用
List = ['a', 'b']

for i, x in tqdm(enumerate(info_all)):
    
    #投稿ページ取得    
    post_text = get_text_from_target_page(driver, True, info_all[x]['url'])
    time.sleep(random.uniform(1, 5))
    
    #70%の確立でいいね→フォローするしないを設定
    if np.random.choice(List, 1, p=[0.70, 0.30])[0] == 'a':
        if post_text:            
            #いいね
            time.sleep(random.uniform(1, 5))
            like = driver.find_element_by_class_name('fr66n')
            like.click()
            #driver.execute_script("arguments[0].click();", like)
            
            #ユーザー名取得
            time.sleep(1)
            user = driver.find_element_by_class_name('Jv7Aj.mArmR.MqpiF')
            user_name = user.text
            
            #ユーザーページ遷移
            time.sleep(random.uniform(1, 5))
            driver.get(f'https://www.instagram.com/{user_name}/')
            
            try:
                #フォロワー数を取得
                time.sleep(1)
                tmp = driver.find_elements_by_css_selector("li.Y8-fY")[1]
                tmp = tmp.text.replace('フォロワー', '')
                tmp = tmp.replace('人', '')
                follower = int(tmp)

                #フォロー数を取得
                time.sleep(1)
                tmp = driver.find_elements_by_css_selector("li.Y8-fY")[2]
                tmp = tmp.text.replace('フォロー中', '')
                tmp = tmp.replace('人', '')
                follow = int(tmp)
            
            #xx万人 対策(フォローしない)
            except:
                follower = 1
                follow = 0
            
            #クールダウン
            time.sleep(random.uniform(2, 5))
            
            #フォロワーよりもフォロー数が多いときだけ
            if follow > follower:

                #更に70%の確立でフォロー
                if np.random.choice(List, 1, p=[0.70, 0.30])[0] == 'a':
                    #ユーザーページに戻ってくる
                    driver.get(info_all[x]['url'])
                    time.sleep(random.uniform(2, 5))

                    #フォローボタン
                    try:
                        elem_button = driver.find_element_by_class_name('bY2yH') 
                        btn_text = elem_button.text
                        #フォローが未だった場合のみ
                        if btn_text.replace('•\n', '') == 'フォローする':
                            time.sleep(random.uniform(1, 5))
                            elem_button.click()
                    except:
                        pass

        #数秒待機で次の投稿ページへ
        time.sleep(random.uniform(2, 5))

driver.quit()
print('completed!')

57it [17:27, 16.93s/it]

In [ ]:
#ワーニングをリセット
warnings.resetwarnings()